### Рассчитываю индекс физического объём для наблюдений

- Считаю цены 2019 года для товаров, у которых логируется масса
- Джоиню их на товаропотоки
- Считаю индексы физического объёма

In [1]:
import polars as pl

In [2]:
trade19 = pl.read_csv('../raw_data/BACI_HS17_V202501/BACI_HS17_Y2019_V202501.csv')
trade20 = pl.read_csv('../raw_data/BACI_HS17_V202501/BACI_HS17_Y2020_V202501.csv')
trade21 = pl.read_csv('../raw_data/BACI_HS17_V202501/BACI_HS17_Y2021_V202501.csv')
trade22 = pl.read_csv('../raw_data/BACI_HS17_V202501/BACI_HS17_Y2022_V202501.csv')

In [3]:
country_codes = pl.read_csv('../raw_data/BACI_HS17_V202501/country_codes_V202501.csv')
sql = pl.SQLContext(register_globals=True)

In [4]:
sql.execute('''select *, v/q as p from trade19 limit 10;''').collect()

t,i,j,k,v,q,p
i64,i64,i64,i64,f64,f64,f64
2019,4,20,420229,0.471,0.001,471.0
2019,4,24,481940,1.426,1.0,1.426
2019,4,24,847141,0.966,0.022,43.909091
2019,4,31,70310,6.404,22.0,0.291091
2019,4,31,80211,2.472,0.196,12.612245
2019,4,31,80620,0.187,0.028,6.678571
2019,4,31,80711,6.513,37.83,0.172165
2019,4,31,81310,0.118,0.014,8.428571
2019,4,31,81340,0.128,0.02,6.4


In [8]:
sql.execute('''select k from trade19 limit 2;''').collect()

k
i64
420229
481940


In [24]:
# Считаю категории, для которых не определена масса перевезённых грузов
category_x_price = sql.execute('''
with categories as (
select *, left(case when length(k::varchar) = 5 then '0'||(k::varchar) 
            else k::varchar end, 2) as category
from trade19)
                        
select 
    category 
    , sum(v) as v
    , sum(q) as q        
    , avg(case when q is not null then 1 else 0 end) as q_not_null_ratio
    , count(*)   
    , avg(v/q) as avg_price                           
from categories
group by  1  
order by 4        
;''').collect()
category_x_price.write_excel('../processed_data/category_x_q_hit.xlsx')

In [29]:
import plotly.express as px
px.scatter(category_x_price, x='q_not_null_ratio', y='avg_price', hover_data='category', log_y=True,
           labels={'q_not_null_ratio':'Доля наблюдений с известной массой', 'avg_price':'Средняя цена тонны груза'})

In [4]:
# Меняю типы данных на более экономные, где это возможно, и объединяю таблицы для трёх лет
trade_tmp = sql.execute(''' 
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade20
            union all
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade21
            union all
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade22
            ;
            ''').collect()
sql.register('trade_tmp', trade_tmp)
sql.execute('''select count(*) from trade_tmp;''').collect()

len
u32
34145929


In [5]:
# Удаляю лишние таблицы, чтобы не засоряли оперативку
import gc
del trade20
del trade21
del trade22
gc.collect()

23

In [6]:
# Джоиню данные за 2019 год, чтобы было потом на них нормировать
trade_tmp2 = sql.execute('''
            select 
                tr.t
                , tr.i
                , tr.j
                , tr.category    
                , tr.v
                , tr.q             
                , t19.v as v19
                , t19.q as q19
                , t19.v/t19.q as p19             
            from trade_tmp tr
            inner join trade19 t19
                on tr.i = t19.i
                and tr.j = t19.j
                and tr.k = t19.k
            ;''').collect()
sql.register('trade_tmp2', trade_tmp2)
del trade19
del trade_tmp
gc.collect()

0

In [7]:
sql.execute('''select * from trade_tmp2 limit 2;''').collect()

t,i,j,category,v,q,v19,q19,p19
i16,i16,i16,str,f32,f32,f64,f64,f64
2020,4,36,"""080211""",2.456,0.71,6.944,1.198,5.796327
2020,4,36,"""080420""",1.398,1.331,0.87,0.8,1.0875


In [5]:
# Материализую
# trade_tmp2.write_parquet('../preprocessed_data/trade_tmp2_with_prices.parquet.gzip')
trade_tmp2 = pl.read_parquet('../preprocessed_data/trade_tmp2_with_prices.parquet.gzip')
sql.register('trade_tmp2', trade_tmp2)

<SQLContext [tables:2] at 0x223309c7e00>

In [7]:
# У некоторых стран два country_code, но один iso3 код. Чтобы убрать дубли, группирую и суммирую
trade = sql.execute('''
    select 
        tr.t
        , cc_i.country_iso3 as i
        , cc_j.country_iso3 as j
        , tr.category    
        , v
        , q             
        , v19
        , q19
        , p19            
    from trade_tmp2 tr
    inner join country_codes cc_i
        on tr.i = cc_i.country_code 
    inner join country_codes cc_j
        on tr.j = cc_j.country_code                      
;''').collect()
sql.register('trade', trade)
sql.execute('''select * from trade limit 2;''').collect()

t,i,j,category,v,q,v19,q19,p19
i16,str,str,str,f32,f32,f64,f64,f64
2020,"""AFG""","""AUS""","""080211""",2.456,0.71,6.944,1.198,5.796327
2020,"""AFG""","""AUS""","""080420""",1.398,1.331,0.87,0.8,1.0875


In [ ]:
sql.execute('''select count(*) from trade where p19 is null limit 2;''').collect()

len
u32
383132


In [11]:
trade_hs0_pvi = sql.execute('''
select         
    t
    , i
    , j    
    , sum(p19*q)/sum(p19*q19) as pvi
from trade 
group by 1, 2, 3 
;''').collect()
sql.register('trade_hs0_pvi', trade_hs0_pvi)
sql.execute('''select * from trade_hs0_pvi limit 2;''').collect()

t,i,j,pvi
i16,str,str,f64
2020,"""BGD""","""SVK""",1.046154
2020,"""BRA""","""TCA""",0.705113


### Джоиню регрессоры

In [12]:
regressors = pl.read_parquet('../preprocessed_data/regressors.parquet.gzip').with_columns(pl.col("t").cast(pl.Utf8).alias("year"))
sql.register('regressors', regressors)
sql.execute('''select * from regressors limit 2;''').collect()

country,t,cases,deaths,tests,C1M_School_closing,C2M_Workplace_closing,C3M_Cancel_public_events,C4M_Restrictions_on_gatherings,C5M_Close_public_transport,C6M_Stay_at_home_requirements,C7M_Restrictions_on_internal_movement,C8EV_International_travel_controls,E1_Income_support,E2_Debt_contract_relief,E3_Fiscal_measures,E4_International_support,H1_Public_information_campaigns,H2_Testing_policy,H3_Contact_tracing,H4_Emergency_investment_in_healthcare,H5_Investment_in_vaccines,H6M_Facial_Coverings,H7_Vaccination_policy,H8M_Protection_of_elderly_people,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,MajorityVaccinated,PopulationVaccinated,country:temp,t:temp,average_surface_temperature,country:median_age_table,t:median_age_table,median_age,country:SARS_table,SARS_total_cases,country:density_and_mortality,t:density_and_mortality,population_density,female_ratio,maternal_mortality,cvd_mortality,pollution_mortality,poisoning_mortality,hygiene_mortality,female_mortality,male_mortality,infant_mortality,largest_city_share,urbanization,country:household_size,avg_hh_size,share_1_members,share_2_3_members,share_4_5_members,year
str,i16,f64,f64,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,i64,f64,str,i64,f32,str,i64,str,i64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,f32,f32,f32,f32,str
"""SLV""",2020,19.625912,0.570608,null,2.426229,1.434426,1.306011,2.297814,0.833333,1.800546,1.295082,2.70765,0.751366,1.445355,4.871233e6,0.0,1.879781,0.808743,0.631148,489863.0,0.0,2.196721,0.0,2.046448,63.594536,53.958168,53.822815,54.918034,0.0,0.0,"""SLV""",2020,26.48445,"""SLV""",2020,25.691,null,null,"""SLV""",2020,300.901207,52.477556,null,null,null,null,null,124.344002,293.265015,11.0,24.148672,17.73575,"""SLV""",3.92,8.46,36.009998,37.84,"""2020"""
"""ASM""",2022,467.620027,1.926,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""ASM""",2022,26.570254,"""ASM""",2022,27.927,null,null,"""ASM""",2022,241.71,49.464234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""2022"""


In [13]:
sql.execute('''create table trade_x_covid as
    select *
    from trade_hs0_pvi tr
    inner join regressors r_i 
        on tr.i = r_i.country
        and tr.t = r_i.t        
    inner join regressors r_j 
        on tr.j = r_j.country
        and tr.t = r_j.t                               
;''')
sql.execute('''select * from trade_x_covid limit 2;''').collect()

t,i,j,pvi,country,t:r_i,cases,deaths,tests,C1M_School_closing,C2M_Workplace_closing,C3M_Cancel_public_events,C4M_Restrictions_on_gatherings,C5M_Close_public_transport,C6M_Stay_at_home_requirements,C7M_Restrictions_on_internal_movement,C8EV_International_travel_controls,E1_Income_support,E2_Debt_contract_relief,E3_Fiscal_measures,E4_International_support,H1_Public_information_campaigns,H2_Testing_policy,H3_Contact_tracing,H4_Emergency_investment_in_healthcare,H5_Investment_in_vaccines,H6M_Facial_Coverings,H7_Vaccination_policy,H8M_Protection_of_elderly_people,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,MajorityVaccinated,PopulationVaccinated,country:temp,t:temp,…,H6M_Facial_Coverings:r_j,H7_Vaccination_policy:r_j,H8M_Protection_of_elderly_people:r_j,StringencyIndex_Average:r_j,GovernmentResponseIndex_Average:r_j,ContainmentHealthIndex_Average:r_j,EconomicSupportIndex:r_j,MajorityVaccinated:r_j,PopulationVaccinated:r_j,country:temp:r_j,t:temp:r_j,average_surface_temperature:r_j,country:median_age_table:r_j,t:median_age_table:r_j,median_age:r_j,country:SARS_table:r_j,SARS_total_cases:r_j,country:density_and_mortality:r_j,t:density_and_mortality:r_j,population_density:r_j,female_ratio:r_j,maternal_mortality:r_j,cvd_mortality:r_j,pollution_mortality:r_j,poisoning_mortality:r_j,hygiene_mortality:r_j,female_mortality:r_j,male_mortality:r_j,infant_mortality:r_j,largest_city_share:r_j,urbanization:r_j,country:household_size:r_j,avg_hh_size:r_j,share_1_members:r_j,share_2_3_members:r_j,share_4_5_members:r_j,year:r_j
i16,str,str,f64,str,i16,f64,f64,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,i64,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,i64,f64,str,i64,f32,str,i64,str,i64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,f32,f32,f32,f32,str
2020,"""BGD""","""SVK""",1.046154,"""BGD""",2020,8.303453,0.121436,null,2.385246,2.065574,1.576503,3.103825,1.128415,1.486339,1.486339,2.491803,0.084699,1.530055,2.5275566e7,4098.36084,1.871585,1.653005,0.937158,354836.90625,0.0,2.360656,0.04918,0.0,66.011719,52.2994,54.002678,40.368851,0.0,0.0,"""BGD""",2020,…,2.245902,0.016393,2.122951,45.505875,52.073471,49.803879,67.964478,0.0,0.0,"""SVK""",2020,9.500336,"""SVK""",2020,40.195999,null,null,"""SVK""",2020,113.536335,51.156851,null,null,null,null,null,null,null,4.9,14.820294,null,"""SVK""",2.9,25.33,41.369999,26.809999,"""2020"""
2020,"""BRA""","""TCA""",0.705113,"""BRA""",2020,97.838812,2.502044,null,2.418033,2.125683,1.612022,2.874317,1.31694,1.480874,1.5847,1.879781,0.740437,0.740437,5.07706816e8,0.0,1.756831,1.519126,1.502732,2.8938184e7,346994.53125,3.060109,0.0,0.781421,57.505627,50.881092,52.861641,37.021858,0.0,0.0,"""BRA""",2020,…,null,null,null,null,null,null,null,null,null,null,null,null,"""TCA""",2020,37.028999,null,null,"""TCA""",2020,46.722105,49.843416,null,null,null,null,null,74.804001,130.348007,4.3,null,null,"""TCA""",2.37,33.59,46.610001,16.780001,"""2020"""


In [14]:
sql.execute('''select * from trade_x_covid;''').collect().write_parquet('../processed_data/trade_pvi_hs0_x_covid.parquet.gzip')